In [2]:
!pip install datasets
!pip install transformers 
!pip install evaluate
!pip install rouge-score
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 29.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 28.3 MB/s eta 0:00

In [3]:
from pathlib import Path
from sklearn.model_selection import train_test_split
from google.colab import drive
from transformers import pipeline
from transformers import AutoTokenizer, DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

import numpy as np
import pandas as pd
import datasets
import re
import time
import math
import evaluate
import torch
import random

# Read in files and subset

In [4]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
%ls

gdrive/  sample_data/


In [6]:
# Read in text and summaries
all_text = pd.read_csv("/content/gdrive/MyDrive/modified_data/text_and_summaries_filtered_split.csv")
all_text.head()

,state_name,state,bill_id,bill_name,original_text,cleaned_text,split_cleaned_text,summary,summary_source,category,status,link,token_list,token_list_lens,split_idx_list,split_text_512,doc_number,split_text
0,Alaska,AK,AK HB27,HB27,HB0027a 1 HB 27 New Text Underlined DEL...,. section . as . is amended by adding new sub...,"['section', 'as', 'is amended by adding new s...",The bill amends an existing regulation of scho...,handwritten,Schools & Education,"Referred to committee, 01/19/2023",https://www.akleg.gov/basis/Bill/Detail/33?Roo...,"[[1375, 1], [38, 1], [19, 21012, 57, 2651, 126...","[1, 1, 8, 11, 1, 1, 8, 1, 6, 1, 8, 12, 9, 7, 8...",[],"['section', 'as', 'is amended by adding new s...",1,section as is amended by adding new subsectio...
1,Alaska,AK,AK HB105,HB105,HB0105a 1 HB 105 New Text Underlined DE...,. section . as . is amended to read . local...,"['section', 'as', 'is amended to read', 'local...",This bill adds to the list of parental rights ...,fast democracy,Schools & Education,"First read and referred to committee, 03/08/2023",https://www.akleg.gov/basis/Bill/Detail/33?Roo...,"[[1375, 1], [38, 1], [19, 21012, 12, 608, 1], ...","[1, 1, 4, 10, 12, 9, 11, 15, 2, 11, 12, 3, 8, ...","[61, 117]","['section', 'as', 'is amended to read', 'local...",1,section as is amended to read local school boa...
2,Alaska,AK,AK HB105,HB105,HB0105a 1 HB 105 New Text Underlined DE...,. section . as . is amended to read . local...,"['section', 'as', 'is amended to read', 'local...",This bill adds to the list of parental rights ...,fast democracy,Schools & Education,"First read and referred to committee, 03/08/2023",https://www.akleg.gov/basis/Bill/Detail/33?Roo...,"[[1375, 1], [38, 1], [19, 21012, 12, 608, 1], ...","[1, 1, 4, 10, 12, 9, 11, 15, 2, 11, 12, 3, 8, ...","[61, 117]",['access to school records by parent foster pa...,2,access to school records by parent foster pare...
3,Alaska,AK,AK HB105,HB105,HB0105a 1 HB 105 New Text Underlined DE...,. section . as . is amended to read . local...,"['section', 'as', 'is amended to read', 'local...",This bill adds to the list of parental rights ...,fast democracy,Schools & Education,"First read and referred to committee, 03/08/2023",https://www.akleg.gov/basis/Bill/Detail/33?Roo...,"[[1375, 1], [38, 1], [19, 21012, 12, 608, 1], ...","[1, 1, 4, 10, 12, 9, 11, 15, 2, 11, 12, 3, 8, ...","[61, 117]",['including the nonviolent resolution or media...,3,including the nonviolent resolution or mediati...
4,Alaska,AK,AK SB96,SB96,SB0096A 1 SB 96 New Text Underlined DEL...,. section . as . is amended to read . local...,"['section', 'as', 'is amended to read', 'local...",This bill adds to the list of parental rights ...,fast democracy,Schools & Education,"First read and referred to committee, 03/08/2023",https://www.akleg.gov/basis/Bill/Detail/33?Roo...,"[[1375, 1], [38, 1], [19, 21012, 12, 608, 1], ...","[1, 1, 4, 10, 12, 9, 11, 15, 2, 11, 12, 2, 8, ...","[62, 118]","['section', 'as', 'is amended to read', 'local...",1,section as is amended to read local school boa...


In [7]:
cols_keep = ['state','bill_id', 'summary', 'doc_number', 'split_text']
all_text = all_text[cols_keep]
all_text.rename({'bill_id':'title',
                 'split_text':'text',
                }, axis='columns', inplace=True)
all_text.head()

<ipython-input-7-437091ae060d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_text.rename({'bill_id':'title',


,state,title,summary,doc_number,text
0,AK,AK HB27,The bill amends an existing regulation of scho...,1,section as is amended by adding new subsectio...
1,AK,AK HB105,This bill adds to the list of parental rights ...,1,section as is amended to read local school boa...
2,AK,AK HB105,This bill adds to the list of parental rights ...,2,access to school records by parent foster pare...
3,AK,AK HB105,This bill adds to the list of parental rights ...,3,including the nonviolent resolution or mediati...
4,AK,AK SB96,This bill adds to the list of parental rights ...,1,section as is amended to read local school boa...


In [10]:
all_text['title'].nunique()*train_size

275.2

## Split into training and testing

In [109]:
import math
random.seed(181111123451)
print("- unique bills:", all_text.title.nunique())
# use 80% of the bills for training and the remaining for test
train_size = .80
num_training_bills = math.floor(all_text['title'].nunique()*train_size)
training_bill_names = random.choices(all_text['title'].unique(), k=num_training_bills)
print("- unique bills in training: ", num_training_bills)
print("\t% of unique bills in training:", train_size)

# train, test = train_test_split(all_text[['text', 'summary', 'title']], test_size = 0.2)
train = all_text.loc[all_text.title.apply(lambda x: x in training_bill_names), ['text', 'summary', 'title']]
test = all_text.iloc[all_text.index.difference(train.index), ]
test = test[['text', 'summary', 'title']]
print("- total number of rows in dataset:", len(all_text))
print("- number of rows in training:", len(train))
print("- number of rows in testing:", len(test))
print("\t% of rows in training:", len(train)/len(all_text))
# print(len(test))


AttributeError: ignored

In [ ]:
# Prepare train and test data frames for merge later
train.columns = ["text", "reference_summary", "bill_id"]
train["dataset_type"] = "train"
test.columns = ["text", "reference_summary", "bill_id"]
test["dataset_type"] = "test"

# Create HuggingFace Objects

In [ ]:
train_dataset = datasets.Dataset.from_dict(train)
test_dataset = datasets.Dataset.from_dict(test)
billsum = datasets.DatasetDict({"train": train_dataset, "test": test_dataset})

# Load Tokenizer

In [ ]:
## PREPROCESSING
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
#The preprocessing function you want to create needs to:

# 1. Prefix the input with a prompt so T5 knows this is a summarization task. 
#    Some models capable of multiple NLP tasks require prompting for specific tasks.
# 2. Use the keyword text_target argument when tokenizing labels.
# 3. Truncate sequences to be no longer than the maximum length set by the max_length parameter.
prefix = "summarize: "
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    # Tokenizes and truncates
    labels = tokenizer(text_target=examples["reference_summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# To apply the preprocessing function over the entire dataset
# You can speed up the map function by setting batched=True to process multiple elements of the dataset at once
tokenized_billsum = billsum.map(preprocess_function, batched=True)

In [ ]:
tokenized_billsum["train"]

In [ ]:
# Create batch of examples using DataCollatorForSeq2Seq. It's better to dynamically pad the sentences to the longest length
# in a batch during collation instead of padding to the model's maximum length
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

# Set Up Evaluation

In [ ]:
## SET UP EVALUATOR

# Load the ROUGE metric
rouge = evaluate.load("rouge")

# Create function that passes predictions and labels to compute the ROUGE metric
# This is what is used for model training
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

# Train Model

In [ ]:
#model.save_weights("/content/gdrive/My Drive/weights.h5")

In [ ]:
%ls

In [ ]:
output_directory = "/content/gdrive/My Drive/modified_data"
model_directory =  "/content/gdrive/My Drive/summarizer_model"

In [101]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

training_args = Seq2SeqTrainingArguments(
    output_dir = output_directory,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    push_to_hub=False, # changed to false
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [102]:
#start = time.time()
#trainer.train()
#end = time.time()
#print((end - start)/60)

In [103]:
# Save the model
trainer.save_model(model_directory)

# Test on all bills

In [104]:
# build the summarizer pipeline
summarizer = pipeline("summarization", model=str(model_directory))
# run the summarizer on every row
all_text.loc[:, 'model_summary'] = all_text.loc[:,'text'].apply(lambda x: summarizer(x)[0]['summary_text'])

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Your max_length is set to 200, but you input_length is only 116. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=58)
Token indices sequence length is longer than the specified maximum sequence length for this model (514 > 512). Running this sequence through the model will result in indexing errors
Your max_length is set to 200, but you input_length is only 102. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)
Your max_length is set to 200, but you input_length is only 118. You might consider decreasing max_length ma

In [111]:
all_text.head()

,state,bill_id,reference_summary,doc_number,text,model_summary
0,AK,AK HB27,The bill amends an existing regulation of scho...,1,section as is amended by adding new subsectio...,section as is amended by adding new subsection...
1,AK,AK HB105,This bill adds to the list of parental rights ...,1,section as is amended to read local school boa...,section as amended to read local school boards...
2,AK,AK HB105,This bill adds to the list of parental rights ...,2,access to school records by parent foster pare...,child under years of age is currently or was p...
3,AK,AK HB105,This bill adds to the list of parental rights ...,3,including the nonviolent resolution or mediati...,the nonviolent resolution or mediation of conf...
4,AK,AK SB96,This bill adds to the list of parental rights ...,1,section as is amended to read local school boa...,section as amended to read local school boards...


In [132]:
# create the joined bill text 
all_text.sort_values(by=['state', 'bill_id', 'doc_number'],
                     ascending=True,
                     inplace=True
                     )
joined_bill_text = all_text.groupby(['state', 'bill_id', 'reference_summary'])['text'].apply(lambda x: ' '.join(x)).reset_index()
# create the joined model summaries
joined_model_summaries = all_text.groupby(['state', 'bill_id'])['model_summary'].apply(lambda x: ' '.join(x)).reset_index()
# add to the first dataframe
joined_text_df = joined_bill_text.merge(joined_model_summaries,
                                        left_on = ['state', 'bill_id'],
                                        right_on = ['state', 'bill_id'],
                                        how='inner')
# indicate whether it was train or test
joined_text_df['dataset_type'] = joined_text_df.bill_id.apply(lambda x: 'train' if x in training_bill_names else 'test')

# create dummy variable doc_id since thats what the rouge score script expects
joined_text_df['doc_number'] = 1

In [134]:
joined_text_df.to_csv("/content/gdrive/My Drive/modified_data/model_and_reference_summaries_split-train-test-fix.csv", index=False)